In [86]:
import os
import time
import random
import re
import pandas as pd
import numpy as np

In [80]:
import jieba

In [81]:
import pickle

In [82]:
def dump_pickle(obj,fname):
    with open(fname,'wb') as file:
        pickle.dump(obj,file)
def load_pickle(fname):
    return pickle.load(open(fname,'rb'))

In [83]:
claim_df = pd.read_pickle('../data/patent_data/processed/claim_df_seg_0630.pkl')

In [88]:
patent_text = claim_df.seg_text.tolist()
patent_text = [' '.join(i) for i in patent_text]

# 向量化 使用tfidf 重新训练 限定词表

In [85]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [89]:
patent_tfidf_obj = TfidfVectorizer(max_features=10000)

In [90]:
patent_tfidf_obj.fit(patent_text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [91]:
dump_pickle(patent_tfidf_obj,'../data/patent_data/processed/patent_tfidf_obj_1wdim_0701.pkl')

In [92]:
patent_all_tfidf_matrix = patent_tfidf_obj.transform(patent_text)

# tfidf 抽取关键词

In [146]:
def topkword_tfidf(text,topk=20,tfidf_obj=patent_tfidf_obj):
    '''
    输入text为空格分隔的词们
    '''
    tfidf_vec = patent_tfidf_obj.transform([text])
    tfidf_vec = tfidf_vec.toarray()[0]
    wordlist = list(patent_tfidf_obj.get_feature_names())
    order = np.argsort(tfidf_vec)[::-1]
    kwd = [wordlist[k] for k in order]
    pattern_none_chinese = re.compile('[0-9a-zA-Z]')
    kwd = list(filter(lambda x:not re.match(pattern_none_chinese,x),kwd))
    return kwd[:topk]

In [129]:
patent_text[0]

'一种 处理器 ， 包括 ： 位于 单个 芯片 封装 内 的 多个 处理 核 ， 其中 每个 处理 核 进一步 包括 ： 指令 取出 逻辑 单元 ， 用于 取出 一个 或 多个 指令 ； 指令 解码 逻辑 单元 ， 用于 解码 一个 或 多个 指令 ； 寄存器 文件 ， 包括 一组 向量 寄存器 ， 每个 向量 寄存器 存储 多个 向量 数据 元素 ； 执行 单元 ， 用于 执行 第一 指令 以 读取 单精度 浮点 值 ， 将 该值 转换 为 双 精度 浮点 值 ， 以及 将 结果 作为 可 变数 目的 N 个 向量 数据 元素 存储 在 寄存器 文件 的 向量 寄存器 中 ， 其中 所述 可变 数目 响应 于 所 执行 的 第一 指令 为 一个 或 多个 ； 其中 所述 执行 单元 用于 执行 第二 指令 以 将 单精度 浮点 值 转换 为 带符号 整数 值 并 将 结果 存储 在 储存 位置 ， 并且 其中 所述 执行 单元 用于 执行 第三 指令 以 将 单精度 浮点 值 转换 为 无 符号 整数 值 并 将 结果 存储 在 储存 位置 。 如权利要求 1 所述 的 处理器 ， 其特征在于 ， 所述 寄存器 文件 包括 一组 物理 寄存器 ， 用于 存储 浮点 值 和 向量 数据 元素 。'

In [131]:
topkword_tfidf(patent_text[2])

['脱硫',
 '速比',
 '耦合',
 '近似',
 '那个',
 '络合物',
 '照相机',
 '偏置',
 '倾斜地',
 '端面',
 '基站',
 '图像编码',
 '波束',
 '灭火',
 '图像处理',
 '炎症',
 '圆环',
 '绝缘体',
 '收纳',
 '还原性']

In [132]:
with open('../data/patent_data/processed/claim_kword_tfidf.txt','w',encoding='utf-8') as file:
    for num,_ in enumerate(claim_df.itertuples()):
        if num % 1000==0:
            print('[{0}] Num:{1}'.format(time.ctime(),num))
        kw = topkword_tfidf(' '.join(claim_df.loc[num,'seg_text']))
        kw = '|'.join(kw)
        file.write(kw+'\n')

[Tue Jul  2 09:47:47 2019] Num:0
[Tue Jul  2 09:48:03 2019] Num:1000
[Tue Jul  2 09:48:18 2019] Num:2000
[Tue Jul  2 09:48:33 2019] Num:3000
[Tue Jul  2 09:48:48 2019] Num:4000
[Tue Jul  2 09:49:03 2019] Num:5000
[Tue Jul  2 09:49:19 2019] Num:6000
[Tue Jul  2 09:49:34 2019] Num:7000
[Tue Jul  2 09:49:49 2019] Num:8000
[Tue Jul  2 09:50:04 2019] Num:9000
[Tue Jul  2 09:50:19 2019] Num:10000
[Tue Jul  2 09:50:35 2019] Num:11000
[Tue Jul  2 09:50:50 2019] Num:12000
[Tue Jul  2 09:51:05 2019] Num:13000
[Tue Jul  2 09:51:20 2019] Num:14000
[Tue Jul  2 09:51:35 2019] Num:15000
[Tue Jul  2 09:51:50 2019] Num:16000
[Tue Jul  2 09:52:06 2019] Num:17000
[Tue Jul  2 09:52:21 2019] Num:18000
[Tue Jul  2 09:52:36 2019] Num:19000
[Tue Jul  2 09:52:52 2019] Num:20000
[Tue Jul  2 09:53:07 2019] Num:21000
[Tue Jul  2 09:53:22 2019] Num:22000
[Tue Jul  2 09:53:37 2019] Num:23000
[Tue Jul  2 09:53:52 2019] Num:24000
[Tue Jul  2 09:54:07 2019] Num:25000
[Tue Jul  2 09:54:23 2019] Num:26000
[Tue Jul  2 09

[Tue Jul  2 10:43:06 2019] Num:219000
[Tue Jul  2 10:43:21 2019] Num:220000
[Tue Jul  2 10:43:36 2019] Num:221000
[Tue Jul  2 10:43:51 2019] Num:222000
[Tue Jul  2 10:44:06 2019] Num:223000
[Tue Jul  2 10:44:21 2019] Num:224000
[Tue Jul  2 10:44:36 2019] Num:225000
[Tue Jul  2 10:44:51 2019] Num:226000
[Tue Jul  2 10:45:07 2019] Num:227000
[Tue Jul  2 10:45:22 2019] Num:228000
[Tue Jul  2 10:45:37 2019] Num:229000
[Tue Jul  2 10:45:52 2019] Num:230000
[Tue Jul  2 10:46:07 2019] Num:231000
[Tue Jul  2 10:46:22 2019] Num:232000
[Tue Jul  2 10:46:38 2019] Num:233000
[Tue Jul  2 10:46:53 2019] Num:234000
[Tue Jul  2 10:47:08 2019] Num:235000
[Tue Jul  2 10:47:23 2019] Num:236000
[Tue Jul  2 10:47:38 2019] Num:237000
[Tue Jul  2 10:47:53 2019] Num:238000
[Tue Jul  2 10:48:08 2019] Num:239000
[Tue Jul  2 10:48:23 2019] Num:240000
[Tue Jul  2 10:48:39 2019] Num:241000
[Tue Jul  2 10:48:54 2019] Num:242000
[Tue Jul  2 10:49:09 2019] Num:243000
[Tue Jul  2 10:49:24 2019] Num:244000
[Tue Jul  2 

[Tue Jul  2 11:37:42 2019] Num:435000
[Tue Jul  2 11:37:57 2019] Num:436000
[Tue Jul  2 11:38:12 2019] Num:437000
[Tue Jul  2 11:38:27 2019] Num:438000
[Tue Jul  2 11:38:42 2019] Num:439000
[Tue Jul  2 11:38:58 2019] Num:440000
[Tue Jul  2 11:39:13 2019] Num:441000
[Tue Jul  2 11:39:28 2019] Num:442000
[Tue Jul  2 11:39:44 2019] Num:443000
[Tue Jul  2 11:39:59 2019] Num:444000
[Tue Jul  2 11:40:14 2019] Num:445000
[Tue Jul  2 11:40:30 2019] Num:446000
[Tue Jul  2 11:40:45 2019] Num:447000
[Tue Jul  2 11:41:00 2019] Num:448000
[Tue Jul  2 11:41:15 2019] Num:449000
[Tue Jul  2 11:41:30 2019] Num:450000
[Tue Jul  2 11:41:45 2019] Num:451000
[Tue Jul  2 11:42:01 2019] Num:452000
[Tue Jul  2 11:42:16 2019] Num:453000
[Tue Jul  2 11:42:31 2019] Num:454000
[Tue Jul  2 11:42:46 2019] Num:455000
[Tue Jul  2 11:43:02 2019] Num:456000
[Tue Jul  2 11:43:17 2019] Num:457000
[Tue Jul  2 11:43:32 2019] Num:458000
[Tue Jul  2 11:43:47 2019] Num:459000
[Tue Jul  2 11:44:02 2019] Num:460000
[Tue Jul  2 

# 加入df中

In [136]:
kwd_list = [i.replace('\n','').split('|') for i in open('../data/patent_data/processed/claim_kword_tfidf.txt').readlines()]

In [139]:
claim_df['kword_tfidf'] = [' '.join(i) for i in kwd_list]

In [149]:
topkword_tfidf(' '.join(claim_df.loc[1,'seg_text']))

['导管',
 '横向',
 '开口',
 '远侧',
 '动脉',
 '静脉',
 '内腔',
 '所述',
 '本体',
 '限定',
 '组件',
 '流体',
 '连通',
 '部分',
 '根据权利要求',
 '其中',
 '至少',
 '外周',
 '末端',
 '切口']

In [150]:
claim_df.loc[1,'seg_text']

['一种',
 '导管',
 '组件',
 '，',
 '包括',
 '：',
 '导管',
 '本体',
 '，',
 '所述',
 '导管',
 '本体',
 '限定',
 '至少',
 '一',
 '第一',
 '内腔',
 '和',
 '第二',
 '内腔',
 '，',
 '所述',
 '导管',
 '本体',
 '包括',
 '远侧',
 '末端',
 '区域',
 '，',
 '所述',
 '远侧',
 '末端',
 '区域',
 '包括',
 '：',
 '至少',
 '一个',
 '静脉',
 '横向',
 '开口',
 '，',
 '所述',
 '至少',
 '一个',
 '静脉',
 '横向',
 '开口',
 '与',
 '所述',
 '第一',
 '内腔',
 '流体',
 '连通',
 '，',
 '所述',
 '至少',
 '一个',
 '静脉',
 '横向',
 '开口',
 '由',
 '通过',
 '所述',
 '导管',
 '本体',
 '外周',
 '的',
 '一成',
 '角度',
 '的',
 '横向',
 '切口',
 '限定',
 '并且',
 '包括',
 '面对',
 '远侧',
 '的',
 '部分',
 '；',
 '以及',
 '至少',
 '一个',
 '动脉',
 '横向',
 '开口',
 '，',
 '所述',
 '至少',
 '一个',
 '动脉',
 '横向',
 '开口',
 '与',
 '所述',
 '第二',
 '内腔',
 '流体',
 '连通',
 '，',
 '所述',
 '至少',
 '一个',
 '动脉',
 '横向',
 '开口',
 '由',
 '通过',
 '所述',
 '导管',
 '本体',
 '外周',
 '的',
 '一成',
 '角度',
 '的',
 '横向',
 '切口',
 '限定',
 '并且',
 '包括',
 '面对',
 '远侧',
 '的',
 '部分',
 '，',
 '所述',
 '至少',
 '一个',
 '动脉',
 '横向',
 '开口',
 '相对',
 '于',
 '所述',
 '至少',
 '一个',
 '静脉',
 '横向',
 '开口',
 '以',
 '大致',
 '非',
 '交错',
 '结构的',
 '方式',
 '被'

In [152]:
claim_df['kword_tfidf'] = claim_df.seg_text.apply(lambda x:topkword_tfidf(' '.join(x)))

In [154]:
claim_df['kword_tfidf'] = claim_df.kword_tfidf.apply(lambda x:' '.join(x))

In [155]:
claim_df

,id,pub_id,info_id,text,seg_text,kword_tfidf
0,0,CN103257847B,claim-info-CN2013,一种处理器，包括：位于单个芯片封装内的多个处理核，其中每个处理核进一步包括：指令取出逻辑单元...,"[一种, 处理器, ，, 包括, ：, 位于, 单个, 芯片, 封装, 内, 的, 多个, ...",浮点 寄存器 指令 向量 执行 存储 文件 元素 单元 多个 转换 结果 用于 解码 储存 ...
1,1,CN103170050B,claim-info-CN2013,一种导管组件，包括：导管本体，所述导管本体限定至少一第一内腔和第二内腔，所述导管本体包括远侧...,"[一种, 导管, 组件, ，, 包括, ：, 导管, 本体, ，, 所述, 导管, 本体, ...",导管 横向 开口 远侧 动脉 静脉 内腔 所述 本体 限定 组件 流体 连通 部分 根据权利...
2,2,CN103414917B,claim-info-CN2013,一种用于从远程地点上传媒体内容的虚拟宽带传送单元，包括：多个调制解调器管理器，每个调制解调器...,"[一种, 用于, 从, 远程, 地点, 上传, 媒体, 内容, 的, 虚拟, 宽带, 传送,...",调制解调器 无线通信 管理器 分组 宽带 虚拟 所述 传送 上传 如权利要求 视频 多个 相...
3,3,CN103199951B,claim-info-CN2013,一种通信系统中的移动台，所述移动台包括：一种装置，所述装置用于向基站发送帧，其中，第一Zad...,"[一种, 通信, 系统, 中, 的, 移动, 台, ，, 所述, 移动, 台, 包括, ：,...",序列 小区 所述 分配 第一 参数 通信 基站 第二 第二块 第一块 其中 移动 系统 多个...
4,4,CN103527484B,claim-info-CN2013,一种回转式压缩机，其压缩作为制冷剂的HFO-1234yf和R32或R41的混合物，其特征在于...,"[一种, 回转式, 压缩机, ，, 其, 压缩, 作为, 制冷剂, 的, HFO, -, 1...",上述 滑动 曲轴 叶片 构成 汽缸 非金属 压缩 柱塞 轴承 偏心 主轴 压缩机 内周 轴部...
5,5,CN103498666B,claim-info-CN2013,三维天线，包括：支撑结构，包括第一面板、第二面板和第三面板；支撑在第一面板中且界定出有孔区域...,"[三维, 天线, ，, 包括, ：, 支撑, 结构, ，, 包括, 第一, 面板, 、, 第...",线圈 天线 三维 面板 支撑 中且 区域 界定 所述 第三 第二 第一 矩形 如权利要求 其...
6,6,CN103183814B,claim-info-CN2013,一种来自生物质资源的聚酯，其主要重复单元为二羧酸单元和二醇单元，其中，所述二羧酸单元是具有脂...,"[一种, 来自, 生物质, 资源, 的, 聚酯, ，, 其, 主要, 重复, 单元, 为, ...",聚酯 官能 生物质 资源 二羧酸 以上 来自 单元 摩尔 权利要求 二醇 比计 原料 化合物...
7,7,CN103367654B,claim-info-CN2013,一种发光元件，包括：第一电极；第二电极；和包含第一电极和第二电极之间的发光物质的第一层，其中...,"[一种, 发光, 元件, ，, 包括, ：, 第一, 电极, ；, 第二, 电极, ；, 和...",半导体 发光 酞菁 第三层 第二层 包含 电极 其中 氧化 羧酸 元件 吸收 物质 第一层 ...
8,8,CN103624740B,claim-info-CN2013,一种电动操纵的手持挤压机的工作方法，其中，当操纵开关时，一个或多个压钳便从起始位置运动到闭合...,"[一种, 电动, 操纵, 的, 手持, 挤压机, 的, 工作, 方法, ，, 其中, ，, ...",压力 挤压机 操纵 手持 其特征为 电动 借助 传感器 规定 挤压 达到 工作 检验 液压 ...
9,9,CN103498704B,claim-info-CN2013,一种双向流冲击式涡轮装置，所述涡轮装置与经过该涡轮装置的双向反向流一起使用，所述涡轮装置具有...,"[一种, 双向, 流, 冲击, 式, 涡轮, 装置, ，, 所述, 涡轮, 装置, 与, 经...",导向 涡轮 所述 转子 第二组 转向 流动 第一组 叶片 流体 装置 至少 限定 通路 出口...


In [156]:
del claim_df['text']
del claim_df['seg_text']

In [158]:
claim_df.to_pickle('../data/patent_data/processed/claim_kword_df_tfidf.pkl')